In [1]:
import pandas as pd
import numpy as np

In [2]:
df_hubo = pd.read_csv('/Users/parkSY/Downloads/hubo_h90_s7.csv')

In [4]:
pop = pd.read_excel('/Users/parkSY/Downloads/실질수요량_최종.xlsx')

In [5]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import random
from sklearn.cluster import KMeans
%matplotlib inline

In [6]:
pop2= pop[['수요지 위도', '수요지 경도']]
df_hubo2 = df_hubo[['위도', '경도']]

In [7]:
from pulp import *

def p_median_pulp(locations, demands, p, demand_demands):
    # 문제 생성
    prob = LpProblem("p-median", LpMinimize)

    # 후보지 선택 변수
    x = LpVariable.dicts("x", locations, 0, 1, LpBinary)

    # 수요지-후보지 할당 변수
    y = LpVariable.dicts("y", (range(len(demands)), locations), 0, 1, LpBinary)

    # 목적 함수: 수요지-후보지 간의 거리 합 (맨해튼 거리 * 수요량)
    prob += lpSum([demand_demands[i] * manhattan_distance(demands[i], locations[j]) for i in range(len(demands)) for j in range(len(locations))])

    # 제약 조건: 각 수요지에 대해 하나의 후보지 선택
    for i in range(len(demands)):
        prob += lpSum([y[i][j] for j in locations]) == 1

    # 제약 조건: 선택된 후보지의 수가 p개
    prob += lpSum([x[j] for j in locations]) == p

    # 제약 조건: 수요지-후보지 할당 변수와 후보지 선택 변수 사이의 관계
    for i in range(len(demands)):
        for j in locations:
            prob += y[i][j] <= x[j]

    # PuLP solver 실행
    prob.solve()

    # 결과 출력
    print("Status:", LpStatus[prob.status])
    print("Objective Value:", value(prob.objective))

    # 선택된 후보지 출력
    selected_facilities = [j for j in locations if value(x[j]) == 1]
    print("Selected Facilities:", selected_facilities)

    # 수요지-후보지 할당 출력
    facility_assignments = [(i, j) for i in range(len(demands)) for j in locations if value(y[i][j]) == 1]
    print("Facility Assignments:")
    for assignment in facility_assignments:
        print("Demand", assignment[0], "- Facility", assignment[1])

def manhattan_distance(coord1, coord2):
    # 두 좌표 간의 맨해튼 거리 계산
    return abs(coord1[0] - coord2[0]) + abs(coord1[1] - coord2[1])

# 입력 데이터 예시
demand_coords = [(37.123, -122.456), (38.789, -121.234), (36.987, -123.789)]
demand_demands = [10, 15, 8]
candidate_coords = [(37.567, -122.789), (38.123, -121.567), (36.789, -123.456)]
p = 2

# p-median 알고리즘 실행
p_median_pulp(candidate_coords, demand_coords, p, demand_demands)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/parkSY/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/r7/p3_265c511l8msk85fmmp83m0000gn/T/b8327286533548ea8d4d62651de31b24-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/r7/p3_265c511l8msk85fmmp83m0000gn/T/b8327286533548ea8d4d62651de31b24-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 74 RHS
At line 88 BOUNDS
At line 102 ENDATA
Problem MODEL has 13 rows, 13 columns and 30 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 13 rows, 12 columns (12 integer (12 of which binary)) and 30 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 0
Cbc0038I Before mini branch and bound, 12 integers at bound fixed and 0 continuous
C

In [21]:
# 최적화 알고리즘으로 얻은 후보지 리스트 찾기
customers = [tuple(x) for x in pop2.to_records(index=False)] 
candidates = [tuple(x) for x in df_hubo2.to_records(index=False)] 
demands  = pop['수요량']
p_value = 80
optimized_candidates = p_median_pulp(candidates, customers, p_value, demands)
print("pulp로 얻은 후보지 리스트:", optimized_candidates)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/parkSY/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/r7/p3_265c511l8msk85fmmp83m0000gn/T/bcdd2e84399241c5acbc2c49459261f7-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/r7/p3_265c511l8msk85fmmp83m0000gn/T/bcdd2e84399241c5acbc2c49459261f7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 19206 COLUMNS
At line 115325 RHS
At line 134527 BOUNDS
At line 153832 ENDATA
Problem MODEL has 19201 rows, 19304 columns and 57511 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.03 seconds
Cgl0004I processed model has 19201 rows, 19303 columns (19303 integer (19303 of which binary)) and 57511 elements
Cbc0045I No integer variables out of 19303 objects (19303 integer) have costs
Cbc0045I branch on satisfied N create fake objective 

In [22]:
optimized_candidates = [(37.5548794, 126.8132172), (37.5571702, 126.8371852), (37.5511883, 126.8366909), (37.5509046, 126.8277171), (37.5574806, 126.8203907), (37.5537258, 126.8734667), (37.5421003, 126.8332498), (37.5312129, 126.8399943), (37.5500816, 126.8522154), (37.5677425, 126.8515819), (37.553739, 126.8154338), (37.5542327, 126.8343602), (37.5534707, 126.8264736), (37.5524126, 126.8261335), (37.5525027, 126.826618), (37.5526781, 126.8266869), (37.5526368, 126.823911), (37.5488626, 126.8237717), (37.5464392, 126.8618156), (37.5518165, 126.8618489), (37.5603932, 126.8559741), (37.5557926, 126.8612688), (37.5561372, 126.8612868), (37.5560663, 126.8614104), (37.5556963, 126.8612006), (37.5555807, 126.861175), (37.5615922, 126.8400554), (37.5596447, 126.8533106), (37.5557076, 126.8361031), (37.5580389, 126.8208883), (37.5705695, 126.8260767), (37.5714532, 126.8298268), (37.5613126, 126.8296933), (37.5671488, 126.8376503), (37.5654659, 126.8370287), (37.5612441, 126.8363749), (37.5611286, 126.8380064), (37.5581965, 126.8319942), (37.5581597, 126.832367), (37.558119, 126.8327528), (37.557895, 126.8312107), (37.5578249, 126.8319384), (37.5575785, 126.8345138), (37.5580519, 126.8334306), (37.5580182, 126.833807), (37.5579809, 126.8341888), (37.5579438, 126.8345705), (37.5576511, 126.83375), (37.5563052, 126.8358709), (37.5473647, 126.8733434), (37.5548364, 126.8712846), (37.5489938, 126.8172232), (37.5389276, 126.8472594), (37.5448825, 126.8368184), (37.5457202, 126.8376159), (37.5489327, 126.8461981), (37.5446344, 126.8469111), (37.5320243, 126.8480172), (37.5430175, 126.8452383), (37.5548283, 126.8181881), (37.5524821, 126.8301367), (37.5539363, 126.82551), (37.5536957, 126.8291896), (37.5511152, 126.8279662), (37.5494969, 126.8262542), (37.5469007, 126.8613819), (37.5468232, 126.8612113), (37.5569376, 126.859244), (37.5591611, 126.8546445), (37.5539289, 126.8626204), (37.5408227, 126.8469041), (37.5402682, 126.8458514), (37.5394581, 126.8463314), (37.5410242, 126.8426559), (37.5394673, 126.8355013), (37.5463453, 126.8491342), (37.5382836, 126.8473935), (37.5482162, 126.8500796), (37.5484338, 126.8491116), (37.5406506, 126.8453885)]

In [23]:
len(set(optimized_candidates))

80

In [24]:
optimized_candidates_df = pd.DataFrame(optimized_candidates,columns= ['위도','경도'] )
result_full3 = optimized_candidates_df.merge(df_hubo, how='left', left_on=['위도', '경도'], right_on=['위도', '경도'])
optimized_candidates_gdf = result_full3[['건물명칭','위도','경도']]
#optimized_candidates_gdf['선정여부'] = 1
optimized_candidates_gdf

,건물명칭,위도,경도
0,동은교회,37.554879,126.813217
1,발산동교회,37.557170,126.837185
2,기독교대한감리회 하늘빛교회,37.551188,126.836691
3,한밀교회,37.550905,126.827717
4,마곡성서침례교회,37.557481,126.820391
...,...,...,...
75,신나는어린이집,37.546345,126.849134
76,석란어린이집,37.538284,126.847393
77,화곡동유치원,37.548216,126.850080
78,보미어린이집,37.548434,126.849112


In [12]:
optimized_candidates_gdf


/var/folders/r7/p3_265c511l8msk85fmmp83m0000gn/T/ipykernel_87632/2829089720.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  optimized_candidates_gdf['선정여부'] = 1


In [14]:
#optimized_candidates_gdf.to_csv('Desktop/optimized_candidate.csv',index = False) 

In [25]:
all_candidates = df_hubo[['건물명칭', '위도', '경도']]

In [39]:
no_candidates = pd.merge(all_candidates , optimized_candidates_gdf, how='outer', indicator=True )

In [41]:
no_candidates = no_candidates.query('_merge == "left_only"').drop(columns=['_merge'])

In [42]:
no_candidates

,건물명칭,위도,경도
3,발음교회,37.551380,126.834397
4,성민교회,37.553770,126.828254
6,대한예수교장로회 경향교회,37.557201,126.851722
9,기독교대한성결교회강서교회,37.545363,126.837197
13,가양1동 주민자치센터,37.569530,126.844774
...,...,...,...
189,강서구립연지노인복지센터,37.552432,126.852660
191,NaN,37.532128,126.847959
193,나나유치원,37.535953,126.857350
194,개구리어린이집,37.539211,126.850696


In [43]:
no_candidates['선정여부'] = 0

In [44]:
optimized_candidates_gdf['선정여부'] = 1

/var/folders/r7/p3_265c511l8msk85fmmp83m0000gn/T/ipykernel_87632/173759850.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  optimized_candidates_gdf['선정여부'] = 1


In [45]:
optimized_candidates_gdf

,건물명칭,위도,경도,선정여부
0,동은교회,37.554879,126.813217,1
1,발산동교회,37.557170,126.837185,1
2,기독교대한감리회 하늘빛교회,37.551188,126.836691,1
3,한밀교회,37.550905,126.827717,1
4,마곡성서침례교회,37.557481,126.820391,1
...,...,...,...,...
75,신나는어린이집,37.546345,126.849134,1
76,석란어린이집,37.538284,126.847393,1
77,화곡동유치원,37.548216,126.850080,1
78,보미어린이집,37.548434,126.849112,1


In [46]:
optimized_candidates_gdf.to_csv('Desktop/optimized_candidate.csv',index = False) 

In [47]:
# 위/아래로 합치기 - 행 기준
candidates_with_col = pd.concat([optimized_candidates_gdf, no_candidates], axis = 0)

In [48]:
candidates_with_col

,건물명칭,위도,경도,선정여부
0,동은교회,37.554879,126.813217,1
1,발산동교회,37.557170,126.837185,1
2,기독교대한감리회 하늘빛교회,37.551188,126.836691,1
3,한밀교회,37.550905,126.827717,1
4,마곡성서침례교회,37.557481,126.820391,1
...,...,...,...,...
189,강서구립연지노인복지센터,37.552432,126.852660,0
191,NaN,37.532128,126.847959,0
193,나나유치원,37.535953,126.857350,0
194,개구리어린이집,37.539211,126.850696,0


In [49]:
candidates_with_col.to_csv('Desktop/candidates_with_col.csv',index = False) 